En este cuaderno se explica el funcionamiento de la clase `CovaltoCsvDataCleane`. Esta clase tiene como propósito limpiar los datos provenientes de un archivo CSV.

Está compuesta por los siguientes métodos:

- __init__: inicializa la clase con la ruta del archivo y verifica que dicho archivo exista. En caso contrario, lanza un error indicando que el archivo no fue encontrado.

- **load_dataset**: Lee el archivo CSV indicado en la ruta de la clase, lo convierte en un DataFrame de pandas y lo retorna para su posterior procesamiento.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

from statsmodels.stats.stattools import medcouple

file_path = lambda file: os.path.join(os.getcwd(),'data/raw',file)

In [ ]:
class CovaltoCsvDataCleane:

    """
    Clase encargada de limpiar los datos provenientes de un archivo csv de la 
    base de datos de covalto.

    Attributes
    ----------
    file_path: str or Path
        Ruta del archivo csv a limpiar
    """

    def __init__(self,file_path):

        """
        función que inicializa la clase con la ruta del archivo y verifica que
        el archivo exista.
        
        Attributes
        ----------
        file_path: Ruta del archivo csv a limpiar

        Raises
        ------
        FileNotFoundError
            Si el archivo específicado no existe

        """

        self.file_path = Path(file_path)
        if not self.file_path.exists():
            raise FileNotFoundError(f"El archivo no existe: {self.file_path}")
        
        self.data = None                    
        

    
    def load_dataset(self):

        """
        Lee el archivo csv indicado en el self.file_path y lo carga como un
        dataframe de pandas.

        return
        ------
        pd.DataFrame
            DataFrame que contiene los datos leidos del archivo csv.

        Raises
        ------
        pandas.errors.EmptyDataError
            Si el archivo está vacío

        pandas.errors.ParserError
            Si el archivo CSV tiene errores de formato o no puede ser parseado correctamente.
        """

        self.data = pd.read_csv(self.file_path)
        return self.data
    
    
    def remove_outliers_medcouple(self, col_interes='ingresos_anuales_mxn',max_retries=3):

        """
        Elimina los valores atípicos de una columna numérica usando el estadístico MedCouple (MC).

        Este método ajusta los límites inferior y superior de detección de outliers 
        en función de la asimetría de la distribución, medida con el estadístico MedCouple.

        Parameters
        ----------
        covalto_dataset_raw: pd.DataFrame
            DataFrame que contiene los datos a procesar.

        col_interes: str 
            Nombre de la columna sobre la cual se eliminaran los valores atípicos.
            Por defecto es 'ingresos_anuales_mxn'.

        max_retries : int
            Número máximo de intentos para ingresar una columna válida.
            Por defecto es 3.
        
        Return
        ------
        pandas.DataFrame
            Copia del DataFrame original sin los valores atípicos en la columna indicada.

        Raises
        ------
        ValueError
            Si no se proporciona una columna válida después de varios intentos.
        
        Notes
        -----
            - Si el MedCouple (MC) es positivo, se ajusta más el límite inferior.
            - Si el MC es negativo, se ajusta más el límite superior.
            - Si el MC es 0, se aplica la regla estándar del rango intercuartílico (IQR).

        """

        if self.data is None:
            raise ValueError("El dataset no ha sido cargado. Ejecute load_dataset() primero.")
        
        attempt = 0
        while attempt<max_retries:
            try:
                # Intenta acceder a la columna
                data = self.data.copy()
                _=data[col_interes]

                # Calcular estadísticas
                resumen = data[col_interes].describe()
                Q1, Q2, Q3 = resumen.iloc[4], resumen.iloc[5], resumen.iloc[6]
                RI = Q3 - Q1
                MC = medcouple(data[col_interes].to_numpy())

                # Determinar límites
                if MC > 0:
                    lim_inf_MC = Q1 - 1.5 * np.exp(-3.5 * MC) * RI
                    lim_sup_MC = Q3 + 1.5 * np.exp(4 * MC) * RI
                elif MC < 0:
                    lim_inf_MC = Q1 - 1.5 * np.exp(-4 * MC) * RI
                    lim_sup_MC = Q3 + 1.5 * np.exp(3.5 * MC) * RI
                else:
                    lim_inf_MC = Q1 - 1.5 * RI
                    lim_sup_MC = Q3 + 1.5 * RI

                data = data[
                    (data[col_interes] >= lim_inf_MC) &
                    (data[col_interes] <= lim_sup_MC) |
                    (data[col_interes].isna())
                ]
                self.data = data.copy()
                return self.data
            
            except KeyError:
                print(f"La columna '{col_interes}' no existe en el DataFrame.")
                attempt += 1
                if attempt < max_retries:
                    col_interes = input("Por favor, ingrese el nombre correcto de la columna: ")
                else:
                    raise ValueError("No se proporcionó una columna válida después de varios intentos.")

    def standardize_sector_column(self,col_name='sector_industrial',max_retries=3):
        """
        Estandariza los nombres del sector industrial en el DataFrame.
        
        Reemplaza valores inconsistentes o en minúsculas por versiones normalizadas.

        Parameters
        ----------
        data : pandas.DataFrame
            DataFrame a procesar.
        
        col_name: str
            Nobre de la columna sobre la cual se estarizarán las categorías
        
        max_retries: int
            Número máximo de intentos para ingregar una columna válida
            Por defecto es 3.
        Returns
        -------
        pandas.DataFrame
            Copia del DataFrame con los nombres del sector estandarizados.
        """
        
        if col_name not in self.data.columns:
            raise ValueError(f"La columna {col_name} no existe en el dataset proporcionado.")
        
        attempt = 0
        while attempt<max_retries:
            try:
                self.data[col_name] = self.data[col_name].replace({
                 'retail': 'Retail'   
                })
                return self.data
            except KeyError:
                print(f"la columna {col_name} no existe en el DataFrame.")
                attempt +=1
                if attempt<max_retries:
                    col_name = print('Ingrese el nombre correcto de la columna:')
                else:
                    ValueError("No se proporcionó una columna válida después de varios intentos")


Leamos el dataframe usando la clase.

In [ ]:
cleaner = CovaltoCsvDataCleane(file_path('covalto_sme_credit_data.csv'))   # Crear instancia
dataset_raw = cleaner.load_dataset()        # Llamar al método
dataset_raw.head()

Limpiemos los valores atípicos usando el método remove_outliers_medcople de la clase.

In [ ]:
# Limpiar outliers
dataset_clean_outliers = cleaner.remove_outliers_medcouple(
    col_interes='ingresos_anuales_mxn',
    max_retries=3
)

In [ ]:
dataset_clean_outliers.shape

In [ ]:
dataset_clean_outliers= cleaner.standardize_sector_column()
dataset_clean_outliers.shape